<a href="https://colab.research.google.com/github/Curiousss/EVAAssignment7/blob/master/TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import tensorflow as tf
import os
import pprint # for pretty printing our device stats

if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    with tf.Session(tpu_address) as session:
      devices = session.list_devices()

    print('TPU devices:')
    pprint.pprint(devices)

TPU address is grpc://10.94.244.122:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 10719749730960052620),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 9751140309556774338),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 3630012588680747504),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 10199871570562237856),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 503829590128109815),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 15127912793016231470),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 11724721852201664558),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 9052286802375921412),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 211398431931

In [20]:
resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.contrib.distribute.initialize_tpu_system(resolver)
strategy = tf.contrib.distribute.TPUStrategy(resolver)

W0729 10:15:01.133288 140278073063296 tpu_strategy_util.py:56] TPU system %s has already been initialized. Reinitializing the TPU can cause previously created variables on TPU to be lost.


In [25]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# preprocessing the dataset
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
print (x_train.shape, y_train.shape)
print (x_test.shape, y_test.shape)

(60000, 28, 28) (60000, 10)
(10000, 28, 28) (10000, 10)


In [0]:
x_train = x_train.reshape(x_train.shape[0], 28, 28,1)
x_test = x_test.reshape(x_test.shape[0], 28, 28,1)

(60000, 28, 28)

In [0]:
with strategy.scope():
  """
	Note: This model can be whatever you want it to be.
	Here, I'm building a simple fully-connected network using 
	our distributed training strategy. 
	This essentailly takes our model and makes it 
	compatible to train on a TPU.
	"""
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Dense(512, input_shape=(784,1), activation='relu'))
  model.add(tf.keras.layers.Dense(256, activation='relu'))
  model.add(tf.keras.layers.Dense(128, activation='relu'))
  model.add(tf.keras.layers.Dense(64, activation='relu'))
  model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
	# compiling the model using the RMSProp optimizer 
	# and Sparse Categorical Crossentropy loss
  model.compile(
        optimizer=tf.train.RMSPropOptimizer(learning_rate=1e-2),
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        metrics=['sparse_categorical_accuracy']
  )
    
  model.summary()

In [38]:
#from tf.keras.layers import Convolution2D, MaxPooling2D, Activation, BatchNormalization, Flatten
with strategy.scope():

  dropout=0.15

  model = tf.keras.models.Sequential()


  model.add(tf.keras.layers.Convolution2D(10, 3, activation='relu', input_shape=(28,28,1))) #26
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(dropout))

  model.add(tf.keras.layers.Convolution2D(16, 3, activation='relu')) #24
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(dropout))

  model.add(tf.keras.layers.Convolution2D(16, 3, activation='relu')) #22
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(dropout))

  # Bottleneck layer
  model.add(tf.keras.layers.MaxPooling2D()) #11
  model.add(tf.keras.layers.Convolution2D(10, 1, activation='relu')) #11
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(dropout))

  model.add(tf.keras.layers.Convolution2D(16, 3, activation='relu')) #9
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(dropout))

  model.add(tf.keras.layers.Convolution2D(16, 3, activation='relu')) #7
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(dropout))

  # 1x1 Conv
  model.add(tf.keras.layers.Convolution2D(10, 1, activation='relu')) #7
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(dropout))

  model.add(tf.keras.layers.Convolution2D(10, 7))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Activation('softmax'))
  model.summary()
  model.compile(
        optimizer=tf.train.AdamOptimizer(learning_rate=1e-2),
        loss=tf.keras.losses.categorical_crossentropy,
        metrics=['accuracy'])


Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 26, 26, 10)        100       
_________________________________________________________________
batch_normalization_34 (Batc (None, 26, 26, 10)        40        
_________________________________________________________________
dropout_34 (Dropout)         (None, 26, 26, 10)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 24, 24, 16)        1456      
_________________________________________________________________
batch_normalization_35 (Batc (None, 24, 24, 16)        64        
_________________________________________________________________
dropout_35 (Dropout)         (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 22, 22, 16)      

In [41]:
history = model.fit(x_train, 
		                y_train,
                    epochs=20,
                    validation_data=(x_test, y_test),
                    steps_per_epoch=50,
                    verbose=1
                    )

model.save_weights('./mnist_model.h5', overwrite=True)

Epoch 1/20
50/50 [==============================] - 30s 601ms/step - loss: 0.0249 - acc: 0.9914 - val_loss: 0.0176 - val_acc: 0.9934
Epoch 2/20
50/50 [==============================] - 27s 538ms/step - loss: 0.0236 - acc: 0.9925 - val_loss: 0.0232 - val_acc: 0.9914
Epoch 3/20
50/50 [==============================] - 29s 580ms/step - loss: 0.0234 - acc: 0.9924 - val_loss: 0.0216 - val_acc: 0.9925
Epoch 4/20
50/50 [==============================] - 30s 599ms/step - loss: 0.0235 - acc: 0.9922 - val_loss: 0.0165 - val_acc: 0.9937
Epoch 5/20
50/50 [==============================] - 32s 634ms/step - loss: 0.0238 - acc: 0.9921 - val_loss: 0.0238 - val_acc: 0.9921
Epoch 6/20
50/50 [==============================] - 33s 664ms/step - loss: 0.0243 - acc: 0.9919 - val_loss: 0.0184 - val_acc: 0.9932
Epoch 7/20
50/50 [==============================] - 35s 702ms/step - loss: 0.0241 - acc: 0.9922 - val_loss: 0.0274 - val_acc: 0.9917
Epoch 8/20
50/50 [==============================] - 36s 721ms/step - 